## Heuristique Gloutonne

In [63]:
import time

def heuristique_gloutonne(jobs):
    t = time.perf_counter()
    
    M = len(jobs)
    J = len(jobs[0])
    
    jobs_done = []
    solution = []
    tps_exec = []
    for m in range(M):
        solution.append([])
        tps_exec.append(0)
        
    for j in range(J):
        
        min_machine = tps_exec.index(min(tps_exec))
        min_tps_exec =  tps_exec[min_machine]

        min_job = 0
        min_job_time = 100000
        for i in range(J):
            if jobs[min_machine][i] < min_job_time and not i in jobs_done:
                min_job_time = jobs[min_machine][i]
                min_job = i
                
        solution[min_machine].append(min_job)
        jobs_done.append(min_job)
        tps_exec[min_machine] += min_job_time

    program_time = time.perf_counter() - t
    
    #print(solution)
    #print(tps_exec)
    return solution, max(tps_exec), program_time

### Formatage de la solution pour l'algo génétique

In [64]:
def convert_solution_for_Gen(matrix_sol, nb_jobs):
    
    converted_sol=[-1 for i in range(nb_jobs)]
    
    for i in range(len(matrix_sol)):
        for job in matrix_sol[i]:
            converted_sol[job]=i
            
    return converted_sol
            

### Génération de la population de départ pour l'algorithme génétique 
> 50% aléatoire , 50% glouton

In [65]:
import random

def generate_population(Matrix, population_size, seed=22):
    
    random.seed(seed)
    
    nb_machines=len(Matrix)
    nb_task=len(Matrix[0])
    
    sol_glout=convert_solution_for_Gen(heuristique_gloutonne(Matrix)[0], nb_task)
    
    pop_heuristique=int(population_size*0.5)
    
    population=[sol_glout for i in range(pop_heuristique)]
    
    pop_random=population_size-pop_heuristique
        
    for i in range(pop_random):
        individual=[]
        for task in range(nb_task):
            rdm_machine=random.randint(0,nb_machines-1)
            individual.append(rdm_machine)
            
        population.append(individual)
    
    return population

### Calcul du score pour un individu de la population
> C'est la durée totale que mettent tous les jobs à s'éxécuter sur les machines indiquées

In [66]:
def score_individual(Matrix, individual):
    nb_machines=len(Matrix)
    score_machine= [0 for i in range(nb_machines)]
    
    for i in range(len(individual)):
        score_machine[individual[i]]+=Matrix[individual[i]][i]
        
    return max(score_machine)

### Récupération de 50% de la population actuelle
>45% sont ceux ayant le meilleur score

>5% sont séléctionnés aléatoirement

In [67]:
def selection_fitness(Matrix, population):
    sorted_pop=sorted(population, key=lambda individual: score_individual(Matrix, individual))
    
    nb_fittest=int(len(population)*0.45)
    nb_random=int(len(population)*0.5)-nb_fittest
    
    parents=sorted_pop[0:nb_fittest]
    
    for i in range(nb_random):
        rdm=random.randint(nb_fittest+1, len(population)-1)
        parents.append(sorted_pop[rdm])
    
    return list(parents)

### Crossover
> Les parents font un accouplement 2 à 2 (Chaleeeuuur)

> L'enfant généré récupère une partie de chacun de ses parents (la taille de chaque partie étant définie aléatoirement)

In [68]:
def crossover(parents):
    
    children=[]
    for i in range(len(parents)):
        p1=parents[i]
        if (i!=len(parents)-1):
            p2=parents[i+1]
        else :
            p2=parents[0]
        
        rdm=random.randint(1,len(p1)-2)
        child = p1[0:rdm] + p2[rdm:]
        
        children.append(child)

    return list(children)

#crossover(selection_fitness(M, population))
        

### Mutations
> 5% de la nouvelle population (parents+enfants) est mutée

> Les mutés subissent un changement sur l'une des valeurs du vecteur (un des jobs ne s'exécute plus sur la même machine)

In [69]:
def mutator(Matrix, population):
    nb_mutated=int(len(population)*0.05)
    for i in range(nb_mutated):
        rdm_individual=random.randint(0, len(population)-1)
        rdm_job=random.randint(0,len(Matrix[0])-1)
        rdm_machine=random.randint(0, len(Matrix)-1)
        
        population[rdm_individual][rdm_job]=rdm_machine
        
    return list(population)

### The fittest
> Pour le résultat à la fin de l'algorithme général

In [70]:
def the_fittest(Matrix, population):
    sorted_pop=sorted(population, key=lambda individual: score_individual(Matrix, individual))
    
    return sorted_pop[0], score_individual(Matrix, sorted_pop[0])

## Algorithme génétique (sisi)

In [76]:
def genetic_algo(Matrix, population_size):
    start = time.time()
    population = generate_population(Matrix, population_size)
    for i in range(500):
        parents = selection_fitness(Matrix, population)
        children = crossover(parents)
        population = mutator(Matrix, parents+children)
        
    solution = the_fittest(Matrix, population)
    
    end=time.time()
    duree=end-start
    
    return solution, duree   

## Formatage de la solution

In [80]:
def format_sol_gen(solution, Matrix):
    
    vecteur_sol=solution[0][0]
       
    #Nombre de Machines
    print(len(Matrix), end='')
    print(",", end='')
            
    #Nombre de Tâches
    print(len(Matrix[0]))
            
    #Separation id_Matrice et Matrice 
    print("---")

    time_vector=[0 for i in range(len(Matrix))]
    
    for v in range(len(vecteur_sol)):
        
        #Numero de la machine
        print(vecteur_sol[v], end='')
        print(",", end='')

        #Numero de la tache
        print(v, end='')
        print(",", end='')

        start = time_vector[vecteur_sol[v]]
        end = start + Matrix[vecteur_sol[v]][v]
        time_vector[vecteur_sol[v]]=end
        
        #Temps du debut de la tache
        print(start, end='')     
        print(",", end='')

        #Temps de la fin de la tache
        print(end)

    #Separation Matrice et Résultats
    print("---")

    #Temps d'execution
    print(solution[1], end='')
    print(",", end='')

    #Resultat Optimal
    print(solution[0][1])

    print()
            
    

In [81]:
import sys

def test_job_scheduling_AlgoGen(inputfile, outputfile):
        
    file = open(outputfile, 'w')

    list_of_matrix = create_matrix_list_from_csv(inputfile)
    id_matrix=0
    for matrix in list_of_matrix:
        if matrix!=[]:
            sol=genetic_algo(matrix, 500)
            format_sol_gen(sol, matrix)

            print(file=file)

    id_matrix+=1

# TEST

In [79]:
M= [[1,2,3,4,5,6,7,8,9],[9,8,7,6,5,4,3,2,1],[2,3,9,8,7,5,1,4,2],
    [7,4,1,8,5,2,9,6,3],[3,6,9,2,5,8,1,4,7],[6,5,4,3,2,1,9,8,7]]
sol = genetic_algo(M,100)
print(sol)

format_sol_gen(sol, M)

(([0, 2, 3, 4, 5, 5, 4, 1, 1], 3), 0.3035011291503906)
6,9
---
0,0,0,1
2,1,0,3
3,2,0,1
4,3,0,2
5,4,0,2
5,5,3,4
4,6,3,4
1,7,0,2
1,8,3,4
---
0.3035011291503906,3

